In [ ]:
#VE
AK = "" ##your ak
SK = "" ##your sk

In [3]:
from volcengine.ApiInfo import ApiInfo
from volcengine.Credentials import Credentials
from volcengine.base.Service import Service
from volcengine.ServiceInfo import ServiceInfo
from volcengine.auth.SignerV4 import SignerV4
from volcengine.base.Request import Request

Domain = "api-knowledgebase.mlp.cn-beijing.volces.com"

def prepare_request(method, path, ak, sk, data=None):
  r = Request()
  r.set_shema("http")
  r.set_method(method)
  r.set_host(Domain)
  r.set_path(path)

  if data is not None:
    r.set_body(json.dumps(data))
  credentials = Credentials(ak, sk, 'air', 'cn-north-1')
  SignerV4.sign(r, credentials)
  return r

def internal_request(method, api, payload, params=None):
  
  req = prepare_request(
                        method = method, 
                        path = api,
                        ak = AK, 
                        sk = SK,
                        data = payload)
  
  r = requests.request(method=req.method,
          url="{}://{}{}".format(req.schema, req.host, req.path),
          headers=req.headers,
          data=req.body,
          params=params,
      )
  return r

<h2>To create KB</h2>

In [16]:
MEMORYKB_NAME = 'bp_memorykb_demo'

In [17]:
import requests
import json

# EntityType (This is the name of the entity schema)
## Required fields: PropertyName, PropertyValueType (enum = ["int64", "list<int64>", "string", "list<string>", "float32", "bool"]), Description, DefaultValue
#### Optional fields: IsPrimaryKey, UseProvided, AggregateExpression

### If you need an AggregateExpression, you need to create a CustomEntityTypeSchema
STUDY_MEMORY = {   
    "EventType": "study_demo",
    "Version": "1",
    "Description": "The assistant is teaching the user and recording all the assistant’s questions and the user’s answers, along with a rating for each answer. If the knowledge point does not exist in knowledge_point_demo, ignore the question. Ensure all fields have values.",
    "Properties": [
        {
            "PropertyName": "knowledge_point_name",
            "PropertyValueType": "string",
            "Description": "The knowledge point collected",
            "DefaultValue": ""
        },
        {
            "PropertyName": "question",
            "PropertyValueType": "string",
            "Description": "The original question from the assistant",
            "DefaultValue": ""
        },
        {
            "PropertyName": "answer",
            "PropertyValueType": "string",
            "Description": "The original answer from the user",
            "DefaultValue": ""
        },
        {
            "PropertyName": "is_user_answered",
            "PropertyValueType": "bool",
            "Description": "Whether the user answered the question",
            "DefaultValue": False
        },
        {
            "PropertyName": "rating",
            "PropertyValueType": "string",
            "Description": "Rating of the user’s answer, can be either 'bad' or 'good'",
            "DefaultValue": "good"
        },
        {
            "PropertyName": "rating_reasoning",
            "PropertyValueType": "string",
            "Description": "Reasoning behind the rating",
            "DefaultValue": ""
        },
        {
            "PropertyName": "issues",
            "PropertyValueType": "list<string>",
            "Description": "Points that need attention, keep up to 3",
            "DefaultValue": []
        },
        {
            "PropertyName": "rating_score",
            "PropertyValueType": "int64",
            "Description": "Score for the user’s answer: 0 = bad, 1 = good",
            "DefaultValue": 0
        },
        {
            "PropertyName": "has_been_taught",
            "PropertyValueType": "int64",
            "Description": "Whether the user has been taught this knowledge point: 1 = yes, 0 = no",
            "DefaultValue": 0
        },
        {
            "PropertyName": "is_answer_good",
            "PropertyValueType": "int64",
            "Description": "Whether the answer is correct: 1 = yes, 0 = no",
            "DefaultValue": 0
        },
        {
            "PropertyName": "is_answer_bad",
            "PropertyValueType": "int64",
            "Description": "Whether the answer is wrong: 1 = yes, 0 = no",
            "DefaultValue": 1
        },
    ],
    "ValidationExpression": "is_user_answered==True"
}

KNOWLEDGE_POINT_DEMO = {  
    "EntityType": "knowledge_point_demo",
    "Version": "1",
    "AssociatedEventTypes": [
        "study_demo"
    ],
    "Role": "user",
    "Description": "Knowledge point",
    "Properties": [
        {
            "PropertyName": "id",
            "PropertyValueType": "string",
            "Description": "Primary key",
            "IsPrimaryKey": True,
            "UseProvided": True,
            "AggregateExpression": None,
            "DefaultValue": None
        },
        {
            "PropertyName": "knowledge_point_desc",
            "PropertyValueType": "string",
            "Description": "Description of the knowledge point (required)",
            "IsPrimaryKey": False,
            "UseProvided": True,
            "AggregateExpression": None,
            "DefaultValue": None
        },
        {
            "PropertyName": "knowledge_point_name",
            "PropertyValueType": "string",
            "Description": "Name of the knowledge point",
            "IsPrimaryKey": True,
            "UseProvided": True,
            "AggregateExpression": None,
            "DefaultValue": None
        },
        {
            "PropertyName": "difficulty_level",
            "PropertyValueType": "list<int64>",
            "Description": "Difficulty level from 1 to 5 (1 = easiest, 5 = hardest, required)",
            "IsPrimaryKey": False,
            "UseProvided": False,
            "AggregateExpression": None,
            "DefaultValue": None
        },
        {
            "PropertyName": "knowledge_start_time",
            "PropertyValueType": "int64",
            "Description": "Time the knowledge point was first taught (timestamp: YYYYMMDDHHMMSS)",
            "DefaultValue": None
        },
        {
            "PropertyName": "rating_score_max",
            "PropertyValueType": "float32",
            "Description": "Maximum rating score",
            "AggregateExpression": {
                "Op": "MAX",
                "EventType": "study_demo",
                "EventPropertyName": "rating_score"
            },
            "DefaultValue": None
        },
        {
            "PropertyName": "has_been_taught",
            "PropertyValueType": "int64",
            "Description": "Whether the user has been taught this knowledge point",
            "AggregateExpression": {
                "Op": "MAX",
                "EventType": "study_demo",
                "EventPropertyName": "has_been_taught"
            },
            "DefaultValue": None
        },
        {
            "PropertyName": "answer_good_count",
            "PropertyValueType": "int64",
            "Description": "Number of correct answers",
            "AggregateExpression": {
                "Op": "SUM",
                "EventType": "study_demo",
                "EventPropertyName": "is_answer_good"
            },
            "DefaultValue": None
        },
        {
            "PropertyName": "answer_bad_count",
            "PropertyValueType": "int64",
            "Description": "Number of wrong answers",
            "AggregateExpression": {
                "Op": "SUM",
                "EventType": "study_demo",
                "EventPropertyName": "is_answer_bad"
            },
            "DefaultValue": None
        },
        {
            "PropertyName": "rating_score_sum",
            "PropertyValueType": "float32",
            "Description": "Total rating score",
            "AggregateExpression": {
                "Op": "SUM",
                "EventType": "study_demo",
                "EventPropertyName": "rating_score"
            },
            "DefaultValue": None
        },
        {
            "PropertyName": "issues",
            "PropertyValueType": "list<string>",
            "Description": "Issues needing attention, keep up to 5",
            "AggregateExpression": {
                "Op": "LLM_MERGE",
                "EventType": "study_demo",
                "EventPropertyName": "issues"
            },
            "DefaultValue": None
        },
        {
            "PropertyName": "count",
            "PropertyValueType": "int64",
            "Description": "Count",
            "AggregateExpression": {
                "Op": "COUNT",
                "EventType": "study_demo",
                "EventPropertyName": "rating_score"
            },
            "DefaultValue": None
        }
    ]
}

# Create collection
path = '/api/memory/collection/create'
playload = {
    'CollectionName': MEMORYKB_NAME,
    'Description': "test description",
    'BuiltinEventTypes': ['sys_common'],
    'CustomEventTypeSchemas': [STUDY_MEMORY],
    'CustomEntityTypeSchemas': [KNOWLEDGE_POINT_DEMO],
}
rsp = internal_request('POST', path, playload)
print(rsp.json())

"""
Example response:
{'ResponseMetadata': {'Action': '', 'Region': 'cn-beijing', 'RequestId': 'xxx', 'Service': 'knowledge_base_server_boe', 'Version': ''}, 'Result': {'resource_id': 'mem-xxxx'}}
"""


{'ResponseMetadata': {'Action': '', 'Region': 'cn-beijing', 'RequestId': '00000000000000007ee9593316aa4c75', 'Service': 'knowledge_base_server', 'Version': ''}, 'Result': {'resource_id': 'mem-7452536f'}}


"\nExample response:\n{'ResponseMetadata': {'Action': '', 'Region': 'cn-beijing', 'RequestId': 'xxx', 'Service': 'knowledge_base_server_boe', 'Version': ''}, 'Result': {'resource_id': 'mem-xxxx'}}\n"

<h2>Created MemoryKB</h2>
{'ResponseMetadata': <br/>
{'Action': '', <br/>
'Region': 'cn-beijing', <br/>
'RequestId': '00000000000000007ee9593316aa4c75', <br/>
'Service': 'knowledge_base_server',<br/>
'Version': ''}, <br/>
'Result': {'resource_id': 'mem-7452536f'}}


previous resource_id created: 'mem-3e35b2ea'



<h2>Upload Conversation</h2>

In [18]:
message_english_study = [
    {
        "role": "assistant",
        "content": "The video just now mentioned pizza, taco, hotdog, and avocado. Which food do you like the best and why?"
    }, {
        "role": "user",
        "content": "I like to eat taco."
    }, {
        "role": "assistant",
        "content": "Great! Tacos are yummy. Have you tried pizza before?"
    }, {
        "role": "user",
        "content": "I don’t know."  
    }, {
        "role": "assistant",
        "content": "Please answer in English. Can you tell me in English?"
    }, {
        "role": "user",
        "content": "I don’t understand the question." 
    }, {
        "role": "assistant",
        "content": "Let me explain. It means 'Have you eaten pizza before?' Now answer in English, ok?"
    }, {
        "role": "user",
        "content": "No."
    }, {
        "role": "assistant",
        "content": "Well done! You did great. Do you like hotdog?"
    }, {
        "role": "user",
        "content": "Yes."
    }, {
        "role": "assistant",
        "content": "Amazing! You're really good at English. Keep learning the following content."
    },
    {
        "role": "assistant",
        "content": "Now try to translate this Chinese sentence into English."
    }, {
        "role": "user",
        "content": "She is angry to her brother."
    }, {
        "role": "assistant",
        "content": "Almost right! The correct preposition is “with”. It's 'She is angry with her brother'.\nHow do you say: 妈妈对弟弟生气了？"
    }, {
        "role": "user",
        "content": "Mom is angry with my brother."
    }, {
        "role": "assistant",
        "content": "Great job! You're doing so well. How do you say: 我奶奶对我生气了？"
    }, {
        "role": "user",
        "content": "Grandmother is angry with me."
    }, {
        "role": "assistant",
        "content": "Fantastic! You're amazing. Keep learning the following content."
    }, {
        "role": "assistant",
        "content": "Now try to translate this Chinese sentence into English.\n他们对迟来的巴士感到生气。How do you say this in English?"
    }, {
        "role": "user",
        "content": "I don't know."
    }, {
        "role": "assistant",
        "content": "Don’t be afraid to make mistakes. The correct answer is: They are angry about the late bus.\nHow do you say: 他们对坏掉的椅子感到生气？"
    }, {
        "role": "user",
        "content": "They are angry about broken chair."
    }, {
        "role": "assistant",
        "content": "Think again — the correct answer is: They are angry about the broken chair.\nHow do you say: 妈妈对吵闹的音乐感到生气？"
    }, {
        "role": "user",
        "content": "Mom is angry about music."
    }, {
        "role": "assistant",
        "content": "Don’t worry about mistakes. The correct answer is: Mom is angry about the noisy music.\nHow do you say: 爸爸对没洗的碗感到生气？"
    }, {
        "role": "user",
        "content": "Dad is angry about 没洗的碗。"  # Not translated yet
    }, {
        "role": "assistant",
        "content": "Please answer in English?"
    }, {
        "role": "user",
        "content": "How do you say 没洗的碗 in English?"  # Asking for translation
    }, {
        "role": "assistant",
        "content": "You can say 'unwashed dishes'. Now try translating: 爸爸对没洗的碗感到生气。How would you say it?"
    }, {
        "role": "user",
        "content": "Dad is angry about unwashed dishes."
    }, {
        "role": "assistant",
        "content": "Awesome! How do you say: 爷爷对没关的电视感到生气？"
    }, {
        "role": "user",
        "content": "Grandpa is angry about the working TV."  # Incorrect translation
    }, {
        "role": "assistant",
        "content": "Great job, you've already used the target language point correctly twice. Keep it up!"
    }
]

path = "/api/memory/messages/add"
playload = {
    "collection_name": MEMORYKB_NAME, 
    "session_id": "message0", 
    "messages": message_english_study, 
    "metadata": {
        "default_user_id": "user1", 
        "default_assistant_id": "assistant1", 
        "time": 1747611158000
    }, 
    'entities': [  ## We can potentially add a layer of LLM to analyze the conversation and extract the entities instead of tagging manually.
        {
            "entity_type": "knowledge_point_demo",
            "entity_scope": [
                {
                    "id": "1",
                    "knowledge_point_name": "taco",
                    "knowledge_point_desc": "Vocabulary"
                }, {
                    "id": "2",
                    "knowledge_point_name": "pizza",
                    "knowledge_point_desc": "Vocabulary"
                }
            ]
        }
    ]
}
rsp = internal_request('POST', path, playload)
print(rsp.json())

"""
Example response:
{'code': 0, 'data': {}, 'message': 'success', 'request_id': 'xxx'}
"""


{'code': 0, 'data': {}, 'message': 'success', 'request_id': '0000000000000000d8fd0697577a4c49'}


"\nExample response:\n{'code': 0, 'data': {}, 'message': 'success', 'request_id': 'xxx'}\n"

<h2>Search Memory</h2>

In [28]:
path = "/api/memory/search"
playload = {
    "collection_name": MEMORYKB_NAME,
    "query": "taco learning result",
    "limit": 5,
    "filter": {
        "user_id": "user1",
        "memory_type": ["study_demo"],
    }
}

rsp = internal_request('POST', path, playload)
print(rsp.json())



{'code': 0, 'data': {'collection_name': 'bp_memorykb_demo', 'count': 2, 'result_list': [{'assistant_id': ['assistant1'], 'group_id': '', 'id': 'MTkzNkBzdHVkeV9kZW1vQG1lc3NhZ2UwXzBfMEA=', 'last_update_logid': '0000000000000000d8fd0697577a4c49', 'memory_info': {'answer': 'I like to eat taco.', 'has_been_taught': 0, 'is_answer_bad': 0, 'is_answer_good': 1, 'is_user_answered': True, 'issues': [], 'knowledge_point_name': 'taco', 'original_messages': '周一, 19-05-2025 07:32 上午\nassistant1(assistant):The video just now mentioned pizza, taco, hotdog, and avocado. Which food do you like the best and why?\nuser1(user):I like to eat taco.', 'question': 'The video just now mentioned pizza, taco, hotdog, and avocado. Which food do you like the best and why?', 'rating': 'good', 'rating_reasoning': '用户清晰表达了喜欢吃taco，回答符合问题要求', 'rating_score': 1}, 'memory_type': 'study_demo', 'primary_key': [''], 'score': 0.4611551761627197, 'session_id': 'message0', 'status': '', 'time': 1747611158000, 'user_id': ['user1

<h2>Delete MemoryKB</h2>

In [ ]:


path = "/api/memory/collection/delete"
payload = {
    "collection_name": MEMORYKB_NAME,
    }


rsp = internal_request('POST', path, payload)
print(rsp.json())



{'ResponseMetadata': {'Action': '', 'Error': {'Code': 'InvalidParameter', 'Message': 'A parameter specified in the request is not valid: collection_name must be 1–64 characters, start with a letter, and contain only letters, digits, or underscores'}, 'Region': 'cn-beijing', 'RequestId': '0000000000000000f44674e07646445a', 'Service': 'knowledge_base_server', 'Version': ''}}
